In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import plotly.express as px

2024-07-30 20:33:59.401958: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Import our input dataset
Spotify =  Path('Resources/spotify_data_trimmed2.csv')
Spotify_df = pd.read_csv(Spotify)
Spotify_df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,3nqQXoyQOWXiESFLlDF1hG,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),Unholy (feat. Kim Petras),100,156943,False,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,4,dance
1,2tTmW7RDtMQtBk7m2rYeSw,Bizarrap;Quevedo,"Quevedo: Bzrp Music Sessions, Vol. 52","Quevedo: Bzrp Music Sessions, Vol. 52",99,198937,False,0.621,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.2300,0.550,128.033,4,hip-hop
2,4uUG5RXrOk84mYEfFvj3cK,David Guetta;Bebe Rexha,I'm Good (Blue),I'm Good (Blue),98,175238,True,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,4,dance
3,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,La Bachata,98,162637,False,0.835,0.679,7,-5.329,0,0.0364,0.58300,0.000002,0.2180,0.850,124.980,4,latin
4,6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny;Chencho Corleone,Un Verano Sin Ti,Me Porto Bonito,97,178567,True,0.911,0.712,1,-5.105,0,0.0817,0.09010,0.000027,0.0933,0.425,92.005,4,latin


In [7]:
final=Spotify_df.select_dtypes(include='number')

In [8]:
clean_df=Spotify_df.loc[:,['artists','popularity','duration_ms','danceability','energy','track_genre']]
clean_df

,artists,popularity,duration_ms,danceability,energy,track_genre
0,Sam Smith;Kim Petras,100,156943,0.714,0.472,dance
1,Bizarrap;Quevedo,99,198937,0.621,0.782,hip-hop
2,David Guetta;Bebe Rexha,98,175238,0.561,0.965,dance
3,Manuel Turizo,98,162637,0.835,0.679,latin
4,Bad Bunny;Chencho Corleone,97,178567,0.911,0.712,latin
...,...,...,...,...,...,...
9995,Oliver Koletzki;Jimmy Sax,59,419500,0.776,0.601,electronic
9996,Joe Hisaishi,59,180333,0.524,0.345,anime
9997,Wang Chung,59,287360,0.728,0.619,synth-pop
9998,Arijit Singh,59,387169,0.512,0.670,pop-film


In [9]:
test = final['popularity'].quantile(0.75)
print(test)

71.0


In [11]:
X = final[["duration_ms", "danceability", "energy","key","valence","instrumentalness","speechiness"  ]] # error was from here
y = (final['popularity'] > final['popularity'].quantile(0.75)).astype(int)





X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Preprocess numerical data for neural network

# Create and fit the StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:


# Define the deep learning model 
nn_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=80, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(units=30, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="sigmoid")
])

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


# Train the model
history = nn_model.fit(X_train_scaled, y_train, epochs=3, validation_split=0.2, verbose=1)


#
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss:.4f}, Accuracy: {model_accuracy*100:.2f}%")




Epoch 1/3


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7344 - loss: 0.5808 - val_accuracy: 0.7644 - val_loss: 0.5304
Epoch 2/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7603 - loss: 0.5364 - val_accuracy: 0.7644 - val_loss: 0.5304
Epoch 3/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7646 - loss: 0.5287 - val_accuracy: 0.7644 - val_loss: 0.5295
63/63 - 0s - 3ms/step - accuracy: 0.7560 - loss: 0.5435
Loss: 0.5435, Accuracy: 75.60%


In [15]:
my_song = pd.DataFrame({
    "duration_ms": [198937],
    "danceability": [0.935],
    "energy": [0.712],
    "key": [2],
    "valence":[0.238],
    "instrumentalness":[0.000005],
    "speechiness" : [0.0864] 
    
})

In [16]:
# Apply the same scaling used during training
my_song_scaled = scaler.transform(my_song)

In [17]:
# Make predictions
predicted_popularity = nn_model.predict(my_song_scaled)

# Print the predicted popularity
print(f"Predicted popularity: {predicted_popularity.item()*100:.1f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted popularity: 37.9%


In [85]:

# Make y_pred
y_pred = (nn_model.predict(X_test) > 0.5).astype(int).flatten()

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      1512
           1       0.00      0.00      0.00       488

    accuracy                           0.76      2000
   macro avg       0.38      0.50      0.43      2000
weighted avg       0.57      0.76      0.65      2000



/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
